# Richards' Equation: Convergence Test #2

* Author: Jhabriel Varela
* E-mail: jhabriel.varela@uib.no
* Date: 05.04.2019
* Institution: PMG - UiB - Norway

## Description of the problem

In this second numerical convergence test of the Richards' Equation, instead of using relatively simple constitutive relationships as in [[1](#ref)], we use the well-known van Genuchten-Mualem retention curves [[2](#ref)] :

$$
\theta(\psi) = \frac{\theta_s - \theta_r}{\left[1 + \left(\alpha |\psi|\right)^n\right]^m} + \theta_s,
$$

$$
k_r^w(\psi) = \frac{\Big\lbrace {1 - \left(\alpha |\psi|\right)^{n-1} \left[1 + \left(\alpha |\psi|\right)^n \right]^{-m} \Big\rbrace }^2}{\left[1 + \left(\alpha |\psi|\right)^n\right]^{m/2}},
$$

where $\alpha$, $n$ and $m$ are correlation parameters, $\theta_s$ is the water content at saturated conditions and $\theta_r$ is the residual water content. For this particular example, we use $\alpha = 0.04$, $n = 2$, $m = 0.5$, $\theta_s = 0.4$ and $\theta_r = 0.1$. Moreover, we neglect gravity contributions.

The convergence analysis is carried out in the same way as we did in [[1](#ref)]. We use the same domain, boundary conditions, initial conditions, analytical solutions, spatial and time refinements and errors. Note that we are not showing the source term nor the Darcy's velocities expressions, since they are simply too long to show them.

## Importing modules

In [21]:
import numpy as np
import scipy.sparse as sps
import porepy as pp
import matplotlib.pyplot as plt

from porepy.ad.forward_mode import Ad_array
np.set_printoptions(precision=5, suppress = True)

## The convergence function

In [22]:
def convergence_richards(cell_num,time_levels):
    
    ## Computing relative permeabilities
    def arithmetic_mpfa_hyd(krw,g,bc,bc_val,psi_m0):
    
        """
        Computes the arithmetic average of the relative permability
        
        SYNOPSIS:
            arithmetic_mpfa_hyd(krw,g,bc_val,psi_m0)
        
        INPUT ARGUMENTS:
            krw         - Lambda function, relative permeability function krw = f(psi)
            g           - PorePy grid object
            bc_val      - NumPy array, containing values of boundary conditions
            psi_m0      - NumPy array, containing values of pressure head at the cell centers
        
        RETURNS:
            krw_ar      - Numpy array, contatining arithmetic averaged relative permeabilities 
                      at the face centers
        """
    
        neu_fcs = bc.is_neu.nonzero()    # neumann boundary faces
        dir_fcs = bc.is_dir.nonzero()    # dirichlet boundary faces
        int_fcs = g.get_internal_faces() # internal faces

        fcs_neigh = np.zeros((g.num_faces,2),dtype=int) #          
        fcs_neigh[:,0] = g.cell_face_as_dense()[0]      # faces neighbouring mapping
        fcs_neigh[:,1] = g.cell_face_as_dense()[1]      # 

        int_fcs_neigh = fcs_neigh[int_fcs]              # internal faces neighbouring mapping

        # Initializing 
        krw_ar = np.zeros(g.num_faces)

        # Neumann boundaries relative permeabilities
        krw_ar[neu_fcs] = 1.

        # Dirichlet boundaries relative permeabilities
        dir_cells_neigh = fcs_neigh[dir_fcs] # neighboring cells of dirichlet faces
        dir_cells = dir_cells_neigh[(dir_cells_neigh >= 0).nonzero()][0] # cells that share a dirichlet face
        krw_ar[dir_fcs] = 0.5 * (krw(bc_val[dir_fcs]) + 
                                 krw(psi_m0[dir_cells]))

        # Internal faces relative permeabilities
        krw_ar[int_fcs] = 0.5 * (krw(psi_m0[int_fcs_neigh[:,0]]) + 
                                 krw(psi_m0[int_fcs_neigh[:,1]]))

        return krw_ar

    ## The source term
    def source_term(g,t):
        
        x = g.cell_centers[0]
        y = g.cell_centers[1]
        
        f = (2*((np.sign(t*x*y*(x - 1)*(y - 1) + 1)*(t*x*y*(x - 1) + t*x*(x - 1)*(y - 1)))/(25*(np.abs(t*x*y*(x - 1)*(y - 1) + 1)**2/625 + 1)**(1/2)) - (np.abs(t*x*y*(x - 1)*(y - 1) + 1)**2*np.sign(t*x*y*(x - 1)*(y - 1) + 1)*(t*x*y*(x - 1) + t*x*(x - 1)*(y - 1)))/(15625*(np.abs(t*x*y*(x - 1)*(y - 1) + 1)**2/625 + 1)**(3/2)))*(np.abs(t*x*y*(x - 1)*(y - 1) + 1)/(25*(np.abs(t*x*y*(x - 1)*(y - 1) + 1)**2/625 + 1)**(1/2)) - 1)*(t*x*y*(x - 1) + t*x*(x - 1)*(y - 1)))/(np.abs(t*x*y*(x - 1)*(y - 1) + 1)**2/625 + 1)**(1/4) + (2*((np.sign(t*x*y*(x - 1)*(y - 1) + 1)*(t*x*y*(y - 1) + t*y*(x - 1)*(y - 1)))/(25*(np.abs(t*x*y*(x - 1)*(y - 1) + 1)**2/625 + 1)**(1/2)) - (np.abs(t*x*y*(x - 1)*(y - 1) + 1)**2*np.sign(t*x*y*(x - 1)*(y - 1) + 1)*(t*x*y*(y - 1) + t*y*(x - 1)*(y - 1)))/(15625*(np.abs(t*x*y*(x - 1)*(y - 1) + 1)**2/625 + 1)**(3/2)))*(np.abs(t*x*y*(x - 1)*(y - 1) + 1)/(25*(np.abs(t*x*y*(x - 1)*(y - 1) + 1)**2/625 + 1)**(1/2)) - 1)*(t*x*y*(y - 1) + t*y*(x - 1)*(y - 1)))/(np.abs(t*x*y*(x - 1)*(y - 1) + 1)**2/625 + 1)**(1/4) + (2*t*x*(np.abs(t*x*y*(x - 1)*(y - 1) + 1)/(25*(np.abs(t*x*y*(x - 1)*(y - 1) + 1)**2/625 + 1)**(1/2)) - 1)**2*(x - 1))/(np.abs(t*x*y*(x - 1)*(y - 1) + 1)**2/625 + 1)**(1/4) + (2*t*y*(np.abs(t*x*y*(x - 1)*(y - 1) + 1)/(25*(np.abs(t*x*y*(x - 1)*(y - 1) + 1)**2/625 + 1)**(1/2)) - 1)**2*(y - 1))/(np.abs(t*x*y*(x - 1)*(y - 1) + 1)**2/625 + 1)**(1/4) - (np.abs(t*x*y*(x - 1)*(y - 1) + 1)*np.sign(t*x*y*(x - 1)*(y - 1) + 1)*(np.abs(t*x*y*(x - 1)*(y - 1) + 1)/(25*(np.abs(t*x*y*(x - 1)*(y - 1) + 1)**2/625 + 1)**(1/2)) - 1)**2*(t*x*y*(x - 1) + t*x*(x - 1)*(y - 1))**2)/(1250*(np.abs(t*x*y*(x - 1)*(y - 1) + 1)**2/625 + 1)**(5/4)) - (np.abs(t*x*y*(x - 1)*(y - 1) + 1)*np.sign(t*x*y*(x - 1)*(y - 1) + 1)*(np.abs(t*x*y*(x - 1)*(y - 1) + 1)/(25*(np.abs(t*x*y*(x - 1)*(y - 1) + 1)**2/625 + 1)**(1/2)) - 1)**2*(t*x*y*(y - 1) + t*y*(x - 1)*(y - 1))**2)/(1250*(np.abs(t*x*y*(x - 1)*(y - 1) + 1)**2/625 + 1)**(5/4)) - (3*x*y*np.abs(t*x*y*(x - 1)*(y - 1) + 1)*np.sign(t*x*y*(x - 1)*(y - 1) + 1)*(x - 1)*(y - 1))/(6250*(np.abs(t*x*y*(x - 1)*(y - 1) + 1)**2/625 + 1)**(3/2))
        
        return f

    ## The analytical solution
    def analytical(g,t):
        
        x_cntr = g.cell_centers[0]
        y_cntr = g.cell_centers[1]
        x_face = g.face_centers[0]
        y_face = g.face_centers[1]
        
        psi = -t*x_cntr*(1-x_cntr)*y_cntr*(1-y_cntr)-1
        
        q_x = ((np.abs(t*x_face*y_face*(x_face - 1)*(y_face - 1) + 1)/(25*(np.abs(t*x_face*y_face*(x_face - 1)*(y_face - 1) + 1)**2/625 + 1)**(1/2)) - 1)**2*(t*x_face*y_face*(y_face - 1) + t*y_face*(x_face - 1)*(y_face - 1)))/(np.abs(t*x_face*y_face*(x_face - 1)*(y_face - 1) + 1)**2/625 + 1)**(1/4)
        q_y = ((np.abs(t*x_face*y_face*(x_face - 1)*(y_face - 1) + 1)/(25*(np.abs(t*x_face*y_face*(x_face - 1)*(y_face - 1) + 1)**2/625 + 1)**(1/2)) - 1)**2*(t*x_face*y_face*(x_face - 1) + t*x_face*(x_face - 1)*(y_face - 1)))/(np.abs(t*x_face*y_face*(x_face - 1)*(y_face - 1) + 1)**2/625 + 1)**(1/4)
        Q = q_x * g.face_normals[0] + q_y * g.face_normals[1]
        
        return psi,Q
    
    ## Setting up the grid
    Nx = Ny = cell_num
    Lx = 1; Ly = 1
    g = pp.CartGrid([Nx,Ny], [Lx,Ly])
    g.compute_geometry()
    V = g.cell_volumes
    
    # Physical parameters
    theta_s = 0.4
    theta_r = 0.1
    alpha = 0.04
    n = 2
    m = 1 - (1/n)
    
    ## Creating the permeability second order tensor          
    perm = pp.SecondOrderTensor(g.dim, np.ones(g.num_cells))
    
    ## Boundary and initial conditions
    b_faces = g.tags['domain_boundary_faces'].nonzero()[0]

    # Extracting indices of boundary faces w.r.t g
    x_min = b_faces[g.face_centers[0,b_faces] < 0.0001]
    x_max = b_faces[g.face_centers[0,b_faces] > 0.9999*Lx]
    y_min = b_faces[g.face_centers[1,b_faces] < 0.0001]
    y_max = b_faces[g.face_centers[1,b_faces] > 0.9999*Ly]
    
    # Extracting indices of boundary faces w.r.t b_faces
    west   = np.in1d(b_faces,x_min).nonzero()
    east   = np.in1d(b_faces,x_max).nonzero()
    south  = np.in1d(b_faces,y_min).nonzero()
    north  = np.in1d(b_faces,y_max).nonzero()
    
    # Setting the tags at each boundary side
    labels = np.array([None]*b_faces.size)
    labels[east]   = 'dir'
    labels[west]   = 'dir'
    labels[south]  = 'dir'
    labels[north]  = 'dir'
    
    # Constructing the bc object
    bc = pp.BoundaryCondition(g, b_faces, labels)
    
    # Constructing the boundary values array
    bc_val = -1 * np.ones(g.num_faces)
    
    ## Creating the data object
    specified_parameters = {"second_order_tensor": perm, 
                        "bc": bc, 
                        "bc_values": bc_val}

    data = pp.initialize_default_data(g,{},"flow", specified_parameters)
    
    ## Performing the discretization using MPFA  
    # Instatiating the pp.Mpfa class
    solver = pp.Mpfa("flow")

    # MPFA discretization
    mpfa_F, mpfa_boundF, _, _ = solver.mpfa(g, perm, bc)
    mpfa_div = pp.fvutils.scalar_divergence(g)
    
    ## Creating MPFA discrete operators
    F      = lambda x: mpfa_F * x
    boundF = lambda x: mpfa_boundF * x
    div    = lambda x: mpfa_div * x
    
    # Water content
    theta = lambda psi: (theta_s-theta_r)/(1 + (alpha*np.abs(psi))**n)**m + theta_s

    # Specific moisture capacity
    C = lambda psi: (-m*n*psi*(theta_s-theta_r)*(alpha*np.abs(psi))**n)/(np.abs(psi)**2 * (1 + (alpha*np.abs(psi))**n)**(m+1)); 

    # Relative permeability
    krw = lambda psi: (1 - (alpha*np.abs(psi))**(n-1) * (1 + (alpha*np.abs(psi))**n)**(-m))**2/(1 + (alpha*np.abs(psi))**n)**(m/2);
    
    # Arithmetic mean of relative permeability
    krw_ar = lambda psi_m: arithmetic_mpfa_hyd(krw,g,bc,bc_val,psi_m)

    # Multiphase Darcy Flux
    q = lambda psi,psi_m: (F(psi) + boundF(bc_val)) * krw_ar(psi_m)

    # Mass Conservation
    psi_eq = lambda psi,psi_n,psi_m,dt: div(q(psi,psi_m)) +  ((psi-psi_m)*C(psi_m) + 
                                                            theta(psi_m) - 
                                                            theta(psi_n)) * (V/dt) - \
                                                            source_term(g,times[tt]) * V
    
    ## Creating AD-object
    psi_ad = Ad_array(-1*np.ones(g.num_cells), sps.diags(np.ones(g.num_cells)))
    
    ## Time parameters
    t0 = 0                                # [s] Initial time
    tf = 1                                # [s] Final simulation time
    tLevels = time_levels                 # [-] Time levels
    times = np.linspace(t0,tf,tLevels+1)  # [s] Vector of time evaluations
    dt = np.diff(times)                   # [s] Vector of time steps

    ## Newton parameters
    newton_param = dict()
    newton_param['max_tol'] = 1E-8            # [cm] maximum absolute tolerance (pressure head)
    newton_param['max_iter'] = 10             # [iter] maximum number of iterations
    newton_param['abs_tol'] = 100             # [cm] absolute tolerance
    newton_param['iter'] = 1                  # [iter] iteration
    
    ## The time loop
    print("\n Performing simulation with h = {} and dt = {}. \n".format(1/Nx,dt[0]))
    
    tt = 0
    while times[tt] < tf:
           
        psi_n = psi_ad.val.copy()                                 # current time level
        newton_param.update({'abs_tol':100, 'iter':1})            # updating tolerance and iterations
        tt += 1
        
        # Newton loop
        while newton_param['abs_tol'] > newton_param['max_tol']   and \
              newton_param['iter']    < newton_param['max_iter']:      
              
            psi_m = psi_ad.val.copy()                                     # current iteration level
            eq = psi_eq(psi_ad,psi_n,psi_m,dt[tt-1])                      # calling discrete equation
            R = eq.val                                                    # determining residual
            J = eq.jac                                                    # determining Jacobian
            y = sps.linalg.spsolve(J,-R)                                  # Newton update
            psi_ad.val +=  y                                              # root for the k-th step
            newton_param['abs_tol'] = np.max(np.abs(psi_ad.val - psi_m))  # checking convergence        

            if newton_param['abs_tol'] <= newton_param['max_tol'] and newton_param['iter'] <= newton_param['max_iter']:
                print('Time: {0:4.4f} [s] \t Iter: {1:1d} \t Error: {2:4.3e} [cm]'.format(times[tt],newton_param['iter'],newton_param['abs_tol']))
            elif newton_param['iter'] > newton_param['max_iter']:
                 print('Error: Newton method did not converge!')
            else:
                 newton_param['iter'] += 1
                    
    ## Computing norm
    
    # Numerical results
    psi_num = psi_ad.val
    Q_num = q(psi_ad.val,psi_m)
    
    # Analytical results
    psi_ex,Q_ex = analytical(g,times[-1])
    
    # Global error
    E = (np.linalg.norm(psi_ex - psi_num)**2 + \
        dt[0] * np.linalg.norm(Q_ex - Q_num)**2)**(0.5)
    
    return E

## Performing the analysis

In [23]:
# Number of cells and time levels
n_cells = np.array([10,20,40,80,160])
t_levels = np.array([10,20,40,80,160])
h = 1/n_cells
tau = 1/t_levels

# Intializing
E = np.zeros(len(n_cells))
E_reduction = np.zeros(len(n_cells)-1)

# Computing the errors
for i in range(len(E)):
    E[i] = convergence_richards(n_cells[i],t_levels[i])

# Computing the error reduction
for i in range(len(E_reduction)):
    E_reduction[i] = E[i]/E[i+1]


 Performing simulation with h = 0.1 and dt = 0.1. 

Time: 0.1000 [s] 	 Iter: 3 	 Error: 2.641e-10 [cm]
Time: 0.2000 [s] 	 Iter: 3 	 Error: 1.059e-09 [cm]
Time: 0.3000 [s] 	 Iter: 3 	 Error: 2.384e-09 [cm]
Time: 0.4000 [s] 	 Iter: 3 	 Error: 4.240e-09 [cm]
Time: 0.5000 [s] 	 Iter: 3 	 Error: 6.627e-09 [cm]
Time: 0.6000 [s] 	 Iter: 3 	 Error: 9.546e-09 [cm]
Time: 0.7000 [s] 	 Iter: 4 	 Error: 1.191e-11 [cm]
Time: 0.8000 [s] 	 Iter: 4 	 Error: 1.779e-11 [cm]
Time: 0.9000 [s] 	 Iter: 4 	 Error: 2.534e-11 [cm]
Time: 1.0000 [s] 	 Iter: 4 	 Error: 3.478e-11 [cm]

 Performing simulation with h = 0.05 and dt = 0.05. 

Time: 0.0500 [s] 	 Iter: 3 	 Error: 3.481e-11 [cm]
Time: 0.1000 [s] 	 Iter: 3 	 Error: 1.397e-10 [cm]
Time: 0.1500 [s] 	 Iter: 3 	 Error: 3.148e-10 [cm]
Time: 0.2000 [s] 	 Iter: 3 	 Error: 5.600e-10 [cm]
Time: 0.2500 [s] 	 Iter: 3 	 Error: 8.754e-10 [cm]
Time: 0.3000 [s] 	 Iter: 3 	 Error: 1.261e-09 [cm]
Time: 0.3500 [s] 	 Iter: 3 	 Error: 1.717e-09 [cm]
Time: 0.4000 [s] 	 Iter: 

## Results

In [24]:
print("Spatial Step:    ",h)
print("Time Step   :    ",tau)
print("Global Error:    ",E)
print("Error Reduction: ",np.concatenate(([np.NaN],E_reduction)))

Spatial Step:     [0.1     0.05    0.025   0.0125  0.00625]
Time Step   :     [0.1     0.05    0.025   0.0125  0.00625]
Global Error:     [0.00486 0.00244 0.00122 0.00061 0.00031]
Error Reduction:  [    nan 1.99395 1.99857 1.99984 2.00004]


## References
<a id='ref'></a>

[1]: <url>https://github.com/jhabriel/pp-implementations/blob/master/unsaturated_flow/convergence_richards/richards_convergence_1.ipynb

[2]: *van Genuchten, M. T. (1980). A closed-form equation for predicting the hydraulic conductivity of unsaturated soils 1. Soil science society of America journal, 44(5), 892-898.*